In [1]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
from math import sqrt
import pandas as pd
import numpy as np
import heapq
import random

In [2]:
n,m=943,1682
cols = ['user_id', 'item_id', 'rating', 'ts']
base = pd.read_csv('../../ml-100k/u1.base',sep='\t',names=cols)
test = pd.read_csv('../../ml-100k/u1.test',sep='\t',names=cols)

In [3]:
# 创建uesr-item矩阵
train_matrix = np.zeros((n, m))
test_matrix = np.zeros((n, m))
y_ui = np.zeros((n, m))
for i in base.itertuples():
    train_matrix[i[1] - 1, i[2] - 1] = i[3]

for i in test.itertuples():
    test_matrix[i[1] - 1, i[2] - 1] = i[3]

# 创建y矩阵
for i in range(n):
    for j in range(m):
        if train_matrix[i,j] != 0:
            y_ui[i,j] = 1

# train_matrix = np.zeros((n, m))
# train_matrix2 = np.zeros((n, m))
# test_matrix = np.zeros((n, m))
# y_ui = np.zeros((n, m))
# TMP = np.array(base)
# co = 0
# total = np.array(TMP).shape[0]
# while co <= total/2:
#     tmp = random.randint(0, np.array(TMP).shape[0])
#     try:
#         train_matrix[int(TMP[tmp][0]) - 1, int(TMP[tmp][1]) - 1] = float(TMP[tmp][2])
#         TMP = np.delete(TMP,[tmp],axis=0)
#         co += 1
#     except:
#         continue
# for i in TMP:
#     train_matrix2[int(i[0]) - 1, int(i[1]) - 1] = float(i[2])

# for i in test.itertuples():
#     test_matrix[i[1] - 1, i[2] - 1] = i[3]

# # 创建y矩阵
# for i in range(n):
#     for j in range(m):
#         if train_matrix[i][j] != 0:
#             y_ui[i][j] = 1

In [4]:
# MFMPC
# init
global_average_rating = train_matrix.sum()/y_ui.sum() 
avg_rating_user = np.array([0.0]*n)
avg_rating_item = np.array([0.0]*m)
for i in range(n):
    if y_ui[i].sum()==0:
        avg_rating_user[i] = global_average_rating
    else:
        avg_rating_user[i] = train_matrix[i].sum()/y_ui[i].sum()
for i in range(m):
    if y_ui[:,i].sum()==0:
        avg_rating_item[i] = global_average_rating
    else:
        avg_rating_item[i] = train_matrix[:,i].sum()/y_ui[:,i].sum()
bias_u = np.array([0.0]*n)  
bias_i = np.array([0.0]*m)
for i in range(n):
    if sum(y_ui[i]) == 0:
        bias_u[i] = 0
    else:
        t = 0
        for j in range(m):
            t += (train_matrix[i,j]-avg_rating_item[j])*y_ui[i,j]
        bias_u[i] = t/sum(y_ui[i])
for i in range(m):
    if sum(y_ui[:,i]) == 0:
        bias_i[i] = 0
    else:
        t = 0
        for j in range(n):
            t += (train_matrix[j,i]-avg_rating_user[j])*y_ui[j,i]
        bias_i[i] = t/sum(y_ui[:,i])

In [5]:
def mfmpc(R, U, V, M, T, d, y_ui,bias_u, bias_i, global_avg_rating, lambd=0.001, gama=0.01):
    minf = 9999999
    Uc = U.copy()
    Vc = V.copy()
    Mc = M.copy()
    bias_uc = bias_u.copy()
    bias_ic = bias_i.copy()
    miu = global_avg_rating
    Uu = np.zeros((n,d))
    for step in tqdm(range(T)):
        fui = 0
        Ut = np.zeros(U.shape)
        Vt = np.zeros(V.shape)
        Mt = np.zeros(M.shape)
        bias_ut = np.zeros(bias_u.shape)
        bias_it = np.zeros(bias_i.shape)
        gar = 0
        count = 0
        countU = np.zeros(bias_u.shape)
        countV = np.zeros(bias_i.shape)
        for u in range(n):
            for k in range(5):
                tmp = len(M[np.where((1+k>=R[u,:]) & (R[u,:]>0+k))])
                if tmp!=0:
                    Uu[u,:] +=sum(M[np.where((1+k>=R[u,:]) & (R[u,:]>0+k))])/sqrt(tmp)
#         plt.imshow(Uu, cmap='coolwarm', origin='upper', aspect="auto")
#         plt.show()
#         break
        for u in range(n):
            for i in range(m):
                if R[u][i] > 0:
                    count += 1
                    countU[u] += 1
                    countV[i] += 1
                    rui = np.dot(U[u,:],V[i,:].T)+np.dot(Uu[u,:],V[i,:].T)+bias_u[u]+bias_i[i]+global_avg_rating
                    if rui > 5:
                        rui = 5
                    elif rui < 1:
                        rui = 1
                    eui = R[u][i] - rui
                    fui += (np.power(eui, 2)+lambd*np.dot(U[u,:], U[u,:].T)+ lambd*np.dot(V[i,:], V[i,:].T)+lambd*np.power(bias_u[u],2)+lambd*np.power(bias_i[i],2)+lambd*np.power(M[np.where(R[u,:]>0)].sum(),2))/2
                    gar -= eui
                    bias_ut[u] -= (eui-lambd* bias_u[u])
                    bias_it[i] -= (eui-lambd* bias_i[i])
                    Ut[u,:] -= (eui*V[i,:]-lambd*U[u,:])
                    Vt[i,:] -= (eui*(U[u,:]+Uu[u,:])-lambd*V[i,:])
                    for k in range(5):
                        if k+1>=rui>k:
                            tmp = len(M[np.where((1+k>=R[u,:]) & (R[u,:]>0+k))])
                            if tmp!=0:
                                Mt[i,:] -= (eui*V[i,:]/sqrt(tmp)-lambd*M[i,:])# gengxin
                                break
        global_avg_rating -= gama*gar/count
        for u in range(n):
            if countU[u] != 0:
                bias_u[u] -= gama*bias_ut[u]/countU[u]
                U[u,:] -= gama*Ut[u,:]/countU[u]
        for i in range(m):
            if countV[i] != 0:
                bias_i[i] -= gama*bias_it[i]/countV[i]
                V[i,:] -= gama*Vt[i,:]/countV[i]
                M[i,:] -= gama*Mt[i,:]/countV[i]
        if minf > fui:
            minf = fui
            Uc = U.copy()
            Vc = V.copy()
            Mc = M.copy()
            bias_uc = bias_u.copy()
            bias_ic = bias_i.copy()
            miu = global_avg_rating
        gama = gama*0.9
    return Uc,Vc,Mc,bias_uc,bias_ic,miu   

In [6]:
# 评估函数
def eva(prediction, truth):
    prediction = prediction[truth.nonzero()].flatten()
    truth = truth[truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, truth)),mean_absolute_error(prediction, truth)

In [7]:
if __name__ == '__main__':
    d = 20
    T = 50
    U = (np.random.rand(n, d)-0.5)*0.01
    V = (np.random.rand(m, d)-0.5)*0.01
    M = (np.random.rand(m, d)-0.5)*0.01
    R = train_matrix.copy()
#     mfmpc(R, U, V, M, T, d, y_ui,bias_u, bias_i, global_average_rating)
    Uf, Vf, Mf,bias_uf,bias_if,miuf = mfmpc(R, U, V, M, T, d, y_ui,bias_u, bias_i, global_average_rating)
    MFMPC = np.ones((n,m))
    for i in range(n):
        for j in range(m):
            MFMPC[i,j] = np.dot(Uf[i,:],Vf[j,:].T)+np.dot(Mf[i,:],Vf[j,:].T)+bias_uf[i]+bias_if[j]+miuf
            if MFMPC[i,j] > 5:
                MFMPC[i,j] = 5
            if MFMPC[i,j] < 1:
                MFMPC[i,j] = 1

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [08:21<00:00, 10.02s/it]


In [8]:
print("MFMPC RMSE、MAE:"+str(eva(MFMPC, test_matrix)))

MFMPC RMSE、MAE:(0.9608626776135201, 0.7591348912961864)
